# ENS-GI Digital Twin: Clinical Parameter Estimation Workflow

## Complete Pipeline for Patient-Specific IBS Diagnosis

This tutorial demonstrates the **full clinical workflow**:

1. **Generate synthetic patient data** (simulating clinical measurements)
2. **PINN parameter estimation** (fast initial estimate)
3. **Bayesian refinement** (uncertainty quantification)
4. **Virtual drug trial** (personalized treatment)
5. **Clinical report generation**

**Use Case:** Gastroenterologist has patient with suspected IBS-C. Use Digital Twin to:
- Estimate underlying biophysical parameters
- Quantify uncertainty in diagnosis
- Predict drug response *before* prescription

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import sys

# Add parent directory to path
sys.path.insert(0, '..')

from ens_gi_core import ENSGIDigitalTwin
from ens_gi_pinn import PINNEstimator, PINNConfig
from ens_gi_bayesian import BayesianEstimator, BayesianConfig
from ens_gi_drug_library import DrugLibrary, apply_drug, VirtualDrugTrial

print("✓ All modules imported successfully")

## Step 1: Generate Synthetic Patient Data

In clinical practice, this would be real EGG (electrogastrography) or HRM (high-resolution manometry) data.
Here we simulate an IBS-C patient with known parameters.

In [ ]:
# Create synthetic IBS-C patient
print("Creating IBS-C patient digital twin...")
patient_twin = ENSGIDigitalTwin(n_segments=12)
patient_twin.apply_profile('ibs_c')

# Known ground truth (hidden from estimators)
true_g_Na = 85.0  # Reduced Na+ conductance
true_g_K = 38.0   # Slightly elevated K+
true_omega = 0.008  # Slower ICC frequency

# Apply known parameters
for neuron in patient_twin.network.neurons:
    neuron.params.g_Na = true_g_Na
    neuron.params.g_K = true_g_K
patient_twin.network.icc.omega = true_omega

# Run simulation (simulates clinical measurement session)
print("Running 2000ms simulation...")
result = patient_twin.run(2000, dt=0.05, I_stim={5: 10.0}, verbose=False)

# Extract biomarkers
baseline_biomarkers = patient_twin.extract_biomarkers()

print("\n📊 Patient Baseline Biomarkers:")
print(f"  Motility Index: {baseline_biomarkers['motility_index']:.3f}")
print(f"  ICC Frequency: {baseline_biomarkers['icc_frequency_cpm']:.2f} cpm")
print(f"  Spike Rate: {baseline_biomarkers['spike_rate_per_neuron']:.3f} Hz/neuron")
print(f"  Mean Voltage: {baseline_biomarkers['mean_voltage']:.2f} mV")
print(f"  Mean Force: {baseline_biomarkers['mean_force']:.4f}")

## Step 2: Visualize Patient Data

In [ ]:
# Plot patient signals
fig, axes = plt.subplots(3, 1, figsize=(12, 8))

t_plot = result['time'][:1000]  # Plot first 1000 points

# Voltage traces
axes[0].plot(t_plot, result['voltages'][:1000, 0], label='Neuron 0', alpha=0.7)
axes[0].plot(t_plot, result['voltages'][:1000, 5], label='Neuron 5', alpha=0.7)
axes[0].set_ylabel('Voltage (mV)')
axes[0].set_title('Neural Activity (IBS-C Patient)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Force traces
axes[1].plot(t_plot, result['forces'][:1000, 0], label='Segment 0', alpha=0.7)
axes[1].plot(t_plot, result['forces'][:1000, 5], label='Segment 5', alpha=0.7)
axes[1].set_ylabel('Force (normalized)')
axes[1].set_title('Smooth Muscle Contractility')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# ICC slow wave
axes[2].plot(t_plot, result['icc_currents'][:1000], color='purple', alpha=0.7)
axes[2].set_ylabel('ICC Current (pA)')
axes[2].set_xlabel('Time (ms)')
axes[2].set_title('ICC Pacemaker Slow Wave (~3 cpm)')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Step 3: PINN Parameter Estimation

Use **Physics-Informed Neural Network** for fast initial parameter estimate.

**Advantages:**
- Fast (~1-2 minutes)
- Handles high-dimensional parameter spaces
- Incorporates physics constraints

In [ ]:
print("\n" + "="*70)
print("PINN PARAMETER ESTIMATION")
print("="*70)

# Create PINN estimator
pinn_config = PINNConfig(
    architecture='resnet',
    hidden_layers=[128, 64, 32],
    learning_rate=0.001,
    batch_size=32,
    physics_weight=0.1
)

# Create clean twin for estimation
twin_for_estimation = ENSGIDigitalTwin(n_segments=12)
pinn = PINNEstimator(
    digital_twin=twin_for_estimation,
    config=pinn_config,
    parameter_names=['g_Na', 'g_K', 'omega']
)

# Generate synthetic training dataset
print("\nGenerating training dataset...")
dataset = pinn.generate_synthetic_dataset(n_samples=500)
print(f"  Generated {len(dataset['parameters'])} samples")

# Train PINN
print("\nTraining PINN (this may take 1-2 minutes)...")
history = pinn.train(
    features=dataset['features'],
    parameters=dataset['parameters'],
    epochs=1000,
    verbose=True
)

# Estimate patient parameters
print("\nEstimating patient parameters with bootstrap uncertainty...")
pinn_estimates = pinn.estimate_parameters(
    voltages=result['voltages'],
    forces=result['forces'],
    calcium=result['calcium'],
    n_bootstrap=50
)

# Display results
print("\n📊 PINN Parameter Estimates:")
print("  Parameter    True Value    Estimated       Error")
print("  " + "-"*55)

for param in ['g_Na', 'g_K', 'omega']:
    if param in pinn_estimates:
        est = pinn_estimates[param]
        true_val = {'g_Na': true_g_Na, 'g_K': true_g_K, 'omega': true_omega}[param]
        error = abs(est['mean'] - true_val) / true_val * 100
        print(f"  {param:<12} {true_val:<13.4f} {est['mean']:.4f} ± {est['std']:.4f}   {error:.1f}%")

## Step 4: Bayesian Refinement

Use **Bayesian MCMC** for uncertainty quantification and credible intervals.

**Advantages:**
- Full posterior distributions
- Credible intervals for clinical decision-making
- Convergence diagnostics (R-hat)

In [ ]:
print("\n" + "="*70)
print("BAYESIAN PARAMETER REFINEMENT")
print("="*70)

# Create Bayesian estimator
bayesian_config = BayesianConfig(
    n_chains=4,
    n_draws=500,
    n_tune=250,
    sampler='NUTS',
    progressbar=True
)

bayes = BayesianEstimator(
    digital_twin=twin_for_estimation,
    config=bayesian_config,
    parameter_names=['g_Na', 'g_K', 'omega']
)

# Run MCMC sampling
print("\nRunning MCMC sampling (this may take 2-3 minutes)...")
try:
    trace = bayes.estimate_parameters(
        observed_voltages=result['voltages'],
        parameter_names=['g_Na', 'g_K', 'omega']
    )

    # Summarize posterior
    posterior_summary = bayes.summarize_posterior(trace)

    # Display results
    print("\n📊 Bayesian Parameter Estimates:")
    print("  Parameter    True Value    Mean ± SD        95% CI             R-hat")
    print("  " + "-"*75)

    for param in ['g_Na', 'g_K', 'omega']:
        if param in posterior_summary:
            post = posterior_summary[param]
            true_val = {'g_Na': true_g_Na, 'g_K': true_g_K, 'omega': true_omega}[param]
            print(f"  {param:<12} {true_val:<13.4f} "
                  f"{post['mean']:.4f} ± {post['std']:.4f}  "
                  f"[{post['ci_lower']:.4f}, {post['ci_upper']:.4f}]  "
                  f"{post.get('rhat', 1.0):.3f}")

    # Compare with PINN
    print("\n📊 PINN vs Bayesian Comparison:")
    comparison = bayes.compare_with_pinn(
        pinn_estimates={k: v['mean'] for k, v in pinn_estimates.items()},
        pinn_uncertainties={k: v['std'] for k, v in pinn_estimates.items()},
        trace=trace
    )

    for param, comp in comparison.items():
        print(f"  {param}: Agreement = {comp['agreement']}, Overlap = {comp['overlap']:.2f}")

except Exception as e:
    print(f"\n⚠ Bayesian estimation failed (this is OK in test environment): {e}")
    print("  Proceeding with PINN estimates only...")

## Step 5: Virtual Drug Trial

Test **Lubiprostone** (ClC-2 activator, FDA-approved for IBS-C) *in silico*.

In [ ]:
print("\n" + "="*70)
print("VIRTUAL DRUG TRIAL: LUBIPROSTONE")
print("="*70)

# Create drug-treated twin
twin_drug = ENSGIDigitalTwin(n_segments=12)
twin_drug.apply_profile('ibs_c')

# Apply estimated parameters
est_g_Na = pinn_estimates['g_Na']['mean']
est_g_K = pinn_estimates['g_K']['mean']

for neuron in twin_drug.network.neurons:
    neuron.params.g_Na = est_g_Na
    neuron.params.g_K = est_g_K

# Apply drug: Lubiprostone 24 mcg (standard dose)
print("\nApplying Lubiprostone 24 mcg...")
drug_info = apply_drug(
    twin_drug,
    DrugLibrary.LUBIPROSTONE,
    dose_mg=0.024,  # 24 mcg
    time_hours=2.0
)

print(f"  Plasma concentration: {drug_info['concentration_uM']:.3f} µM")
print(f"  Modified parameters: {drug_info['modified_parameters']}")

# Run post-drug simulation
twin_drug.run(2000, dt=0.05, I_stim={5: 10.0}, verbose=False)
drug_biomarkers = twin_drug.extract_biomarkers()

# Compare baseline vs drug
print("\n📊 Treatment Response:")
print("  Biomarker              Baseline    Post-Drug   Change")
print("  " + "-"*60)

for key in ['motility_index', 'icc_frequency_cpm', 'spike_rate_per_neuron']:
    baseline = baseline_biomarkers[key]
    post_drug = drug_biomarkers[key]
    change = (post_drug - baseline) / baseline * 100 if baseline != 0 else 0
    print(f"  {key:<23} {baseline:>9.3f}   {post_drug:>9.3f}   {change:>+6.1f}%")

# Clinical interpretation
motility_improvement = (drug_biomarkers['motility_index'] - baseline_biomarkers['motility_index']) / baseline_biomarkers['motility_index']

print("\n💊 Clinical Interpretation:")
if motility_improvement > 0.15:
    print("  ✓ RESPONDER: Significant motility improvement (>15%)")
    print("  Recommendation: Prescribe Lubiprostone 24 mcg BID")
elif motility_improvement > 0.05:
    print("  ~ PARTIAL RESPONDER: Moderate improvement (5-15%)")
    print("  Recommendation: Consider trial, monitor response")
else:
    print("  ✗ NON-RESPONDER: Minimal improvement (<5%)")
    print("  Recommendation: Consider alternative therapy")

## Step 6: Generate Clinical Report

In [ ]:
print("\n" + "="*70)
print("CLINICAL REPORT")
print("="*70)

report = patient_twin.clinical_report()
print(report)

# Add parameter estimates to report
print("\n📊 Estimated Biophysical Parameters:")
print(f"  g_Na: {pinn_estimates['g_Na']['mean']:.2f} ± {pinn_estimates['g_Na']['std']:.2f} mS/cm²")
print(f"  g_K:  {pinn_estimates['g_K']['mean']:.2f} ± {pinn_estimates['g_K']['std']:.2f} mS/cm²")
print(f"  ω:    {pinn_estimates['omega']['mean']:.4f} ± {pinn_estimates['omega']['std']:.4f} rad/ms")

print("\n💊 Recommended Treatment:")
print("  Drug: Lubiprostone (Amitiza®) 24 mcg PO BID")
print(f"  Predicted Response: {motility_improvement*100:.1f}% motility improvement")
print("  Alternative: Linaclotide if ClC-2 agonist fails")

print("\n📅 Follow-up:")
print("  - Reassess symptoms at 4 weeks")
print("  - Repeat HRM if no improvement")
print("  - Consider dietary intervention (FODMAP)")

## Summary

This workflow demonstrated:

1. ✅ **Synthetic patient data generation** (IBS-C profile)
2. ✅ **PINN parameter estimation** (g_Na, g_K, ω)
3. ✅ **Bayesian uncertainty quantification** (95% credible intervals)
4. ✅ **Virtual drug trial** (Lubiprostone response prediction)
5. ✅ **Clinical report generation** (diagnosis + treatment recommendation)

### Key Advantages:

- **Personalized Medicine:** Patient-specific parameter estimation
- **Drug Screening:** Test multiple drugs *before* prescription
- **Uncertainty Quantification:** Credible intervals for clinical decisions
- **Mechanistic Understanding:** Not just correlation, but causal modeling

### Next Steps:

- Integrate with real clinical data (EGG, HRM)
- Validate against patient cohorts
- Expand drug library (currently 7 FDA-approved drugs)
- Clinical trials for prospective validation